In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from hera_cal import redcal
import linsolve

In [2]:
Nants = 37

ants = np.loadtxt('antenna_positions_%d.dat'%Nants)
antpos = {k:v for k,v in zip(range(Nants),ants)}
reds = redcal.get_reds(antpos)

In [3]:
def calibrate(data,gains):
    calib_data = {}
    for (i, j, pol) in data.keys():
        calib_data[(i, j, pol)] = data[(i, j, pol)]/(gains[(i,'Jxx')] * np.conj(gains[(j,'Jxx')]))
    return calib_data

def average_red_vis(data, gains, subreds):
    sub_bl_visib = calibrate(data, gains)
    vis = 0
    for subbl in subreds:
        vis += sub_bl_visib[subbl]/len(subreds)
    return vis

def optimize_red_vis(data, gains, subreds):
    eqs = {}; const = {}
    for (i,j,pol) in subreds:
        eqs['g_%d * g_%d_ * V'%(i,j)] = data[(i,j,pol)]

    for i in range(37):
        const['g_%d'%(i)] = gains[(i,'Jxx')]    

    lps = linsolve.LinearSolver(eqs, **const)
    X = lps.solve()
    return X['V']